In [1]:
#Import of libraries and data preprocessing.
!python3 -m pip install langchain pymongo 

import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(f"Loaded API Key starts with: {OPENAI_API_KEY[:8]}")

Defaulting to user installation because normal site-packages is not writeable
Loaded API Key starts with: sk-proj-


In [2]:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4.1-mini")

/Users/davidahn/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [4]:
import fitz

doc = fitz.open("../riskassessmenttest.pdf") 
text = ""

for i, page in enumerate(doc):
    text += page.get_text()

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_text(text) 

documents

['Investor Profile: Aggressive \n \nThis investment profile is designed for individuals who are comfortable with market volatility and \nare focused on maximizing long-term growth. You have demonstrated a strong tolerance for \nrisk, a preference for higher-growth opportunities, and the ability to stay invested through market \ndownturns. \n \nInvestment Strategy: \nAggressive investors should prioritize growth-oriented assets such as equities, growth stocks, \nsector-focused ETFs, and emerging market investments. A significant portion of the portfolio \ncan be allocated to higher-risk, higher-reward opportunities, as short-term fluctuations are less \nof a concern given your longer investment horizon. \n \nPortfolio Construction: \nA diversified equity-heavy portfolio is recommended, with exposure across multiple industries \nand geographies. Including a small allocation to alternative investments such as real estate',
 'investment trusts (REITs) or innovation-focused funds can furthe

In [6]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

index_name = "rag-first"

pinecone = PineconeVectorStore.from_texts(
    documents, embeddings, index_name=index_name
)

/Users/davidahn/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from pymongo import MongoClient
from typing import Optional

MONGODB_URI = os.getenv("MONGO_URI")  
DB_NAME = os.getenv("MONGODB_DB")     
COLLECTION_NAME = os.getenv("MONGODB_COLLECTION")  

client = MongoClient(MONGODB_URI)
db = client[DB_NAME]
users_collection = db[COLLECTION_NAME]

def get_investing_type(email: str) -> Optional[str]:
    user = users_collection.find_one({"email": email}, {"risk_category": 1, "_id": 0})
    if user:
        return user["risk_category"]
    return None


In [8]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know". Keep in mind that the user's investing type is {investType}

Context: {context}

Question: {question}

"""

prompt = ChatPromptTemplate.from_template(template)
chain = (
    {"context": pinecone.as_retriever(), "question": RunnablePassthrough(), "investType": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

user_email = "placeholder" #this should be retrived from the login session, either from frontend or the authorization apge
question = "What investment strategy should I follow?"

investing_type = get_investing_type(user_email)

if investing_type is None:
    response = "I don't know"
else:
    response = chain.invoke({
        "question": question,
        "investType": investing_type
    })

print(response)


I don't know
